# Formulário avançado do corredor

Este notebook coleta informações detalhadas do atleta em um formato compatível com o `UserProfile` e com as rotinas de geração de plano (`PlanGenerator`). O fluxo segue recomendações de treinadores como Jack Daniels para entender disponibilidade, experiência, objetivos e restrições antes de gerar o plano.

1. Preencha os widgets com os dados do corredor.
2. Clique em **Gerar perfil + JSON** para visualizar o schema salvo.
3. Use **Gerar plano** para criar um plano com os dados coletados (usa `PlanGenerator` e `TrainingZones`).



In [22]:
# Clone o repositório com todos os arquivos necessários
!git clone https://github.com/tallesmedeiros/DecisionMaking.git
%cd DecisionMaking

# Verificar se os arquivos foram carregados
!ls -la *.py

Cloning into 'DecisionMaking'...
remote: Enumerating objects: 301, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 301 (delta 41), reused 1 (delta 1), pack-reused 208 (from 2)
Receiving objects: 100% (301/301), 418.03 KiB | 9.50 MiB/s, done.
Resolving deltas: 100% (151/151), done.
/content/DecisionMaking/DecisionMaking/DecisionMaking/DecisionMaking/DecisionMaking/DecisionMaking
-rw-r--r-- 1 root root  13601 Nov 27 16:38 cli.py
-rw-r--r-- 1 root root   9461 Nov 27 16:38 environment_analysis.py
-rw-r--r-- 1 root root   2085 Nov 27 16:38 example_with_zones.py
-rw-r--r-- 1 root root  14605 Nov 27 16:38 intervals_integration.py
-rw-r--r-- 1 root root  25740 Nov 27 16:38 notebook_widgets.py
-rw-r--r-- 1 root root  16937 Nov 27 16:38 pdf_export.py
-rw-r--r-- 1 root root 102749 Nov 27 16:38 plan_generator.py
-rw-r--r-- 1 root root   6359 Nov 27 16:38 plot_utils.py
-rw-r--r-- 1 root root  42901 Nov 27 16:38 running_plan.py
-rw-r

In [23]:

import json
from datetime import date, datetime

import ipywidgets as widgets
from IPython.display import display, clear_output

from user_profile import UserProfile, RaceGoal
from training_zones import TrainingZones
from plan_generator import PlanGenerator

print("Widgets prontos. Preencha os campos e gere o perfil quando terminar.")



Widgets prontos. Preencha os campos e gere o perfil quando terminar.


## Seções do formulário

Os campos abaixo cobrem todos os atributos usados pelo `PlanGenerator`:
- **Dados pessoais e fisiológicos**: sexo, idade, peso, altura, FC máxima/repouso (usados para estimativas de carga).
- **Experiência e volume**: anos correndo, volume atual/médio/pico e aderência (alimentam cálculo de progressão e risco).
- **Disponibilidade**: dias por semana, horas por dia, blocos críticos e preferência para longão (influenciam agenda e duração de treinos).
- **Objetivos e provas**: prova principal e testes (`RaceGoal`) determinam meta, semanas e ritmo.
- **Ritmos/Zonas**: tempos recentes de prova alimentam `TrainingZones` via método Jack Daniels ou Critical Velocity.
- **Preferências de sessão**: intervalados, tempo run, longão, cross training e mistura de zonas.
- **Lesões e restrições**: definem ajustes, limitações de impacto, zonas vermelhas e rotinas de força.
- **Logística de sessão**: aquecimento, desaquecimento e deslocamento para limitar duração total.



In [24]:
# --- Widgets: dados pessoais ---
name_w = widgets.Text(description="Nome", value="Corredor(a)")
age_w = widgets.BoundedIntText(description="Idade", value=30, min=12, max=90)
weight_w = widgets.FloatText(description="Peso (kg)", value=70.0)
height_w = widgets.FloatText(description="Altura (cm)", value=175.0)
gender_w = widgets.Dropdown(description="Sexo", options=[("Masculino", "M"), ("Feminino", "F"), ("Não especificar", "")])
hr_rest_w = widgets.IntText(description="FC Repouso", value=0)
hr_max_w = widgets.IntText(description="FC Máx", value=0)

# --- Experiência e volume ---
years_w = widgets.FloatSlider(description="Anos correndo", value=2.0, min=0, max=30, step=0.5)
current_km_w = widgets.FloatSlider(description="Km atual/sem", value=35, min=0, max=200, step=5)
avg_km_w = widgets.FloatSlider(description="Média recente (km)", value=40, min=0, max=250, step=5)
peak_km_w = widgets.FloatSlider(description="Pico recente (km)", value=60, min=0, max=300, step=5)
consistent_days_w = widgets.BoundedIntText(description="Dias mantidos", value=4, min=0, max=7)
experience_w = widgets.Dropdown(description="Nível", options=[("Iniciante", "beginner"), ("Intermediário", "intermediate"), ("Avançado", "advanced")], value="intermediate")
tolerated_w = widgets.SelectMultiple(description="Treinos tolerados", options=UserProfile.TOLERATED_WORKOUT_OPTIONS, layout=widgets.Layout(width="400px"))
adherence_w = widgets.BoundedIntText(description="Aderência %", value=85, min=0, max=100)

# --- Disponibilidade ---
days_week_w = widgets.BoundedIntText(description="Dias/sem", value=4, min=1, max=7)
hours_day_w = widgets.FloatSlider(description="Horas/dia", value=1.0, min=0.5, max=4.0, step=0.5)
preferred_time_w = widgets.Dropdown(description="Horário", options=["", "manhã", "tarde", "noite"], value="")
preferred_location_w = widgets.SelectMultiple(description="Locais", options=["rua", "trilha", "pista", "esteira"])
preferred_days_w = widgets.SelectMultiple(description="Dias preferidos", options=PlanGenerator.DAYS_OF_WEEK)
# Substituir campos abertos de blocos críticos por seleções de horários
stressful_blocks_select_w = widgets.SelectMultiple(description="Blocos críticos", options=["Nenhum", "Manhã", "Tarde", "Noite"], value=["Nenhum"])
long_run_days_w = widgets.SelectMultiple(description="Dias para longão", options=PlanGenerator.DAYS_OF_WEEK)
use_alt_weeks_w = widgets.Checkbox(description="Alternar semanas A/B", value=False)
alt_blocks_select_w = widgets.SelectMultiple(description="Blocos críticos B", options=["Nenhum", "Manhã", "Tarde", "Noite"], value=["Nenhum"])
alt_long_days_select_w = widgets.SelectMultiple(description="Longão semana B", options=PlanGenerator.DAYS_OF_WEEK)
# Simplificar grade semanal em uma seleção de padrões
weekly_schedule_select_w = widgets.Dropdown(description="Grade semanal", options=["nenhum", "manhãs", "tardes", "noites", "flexível"], value="nenhum")

# --- Objetivos (RaceGoal) ---
main_distance_w = widgets.Dropdown(description="Prova principal", options=["5K", "10K", "Half Marathon", "Marathon"], value="10K")
main_date_w = widgets.DatePicker(description="Data da prova", value=date.today())
main_name_w = widgets.Dropdown(description="Tipo de prova", options=["Prova oficial", "Corrida local", "Treino controlado", "Outro"], value="Prova oficial")
main_city_w = widgets.Dropdown(description="Local", options=["Belo Horizonte","São Paulo","Rio de Janeiro","Brasília","Outro"], value="Belo Horizonte")
# separar tempo meta em horas, minutos, segundos
main_target_hours_w = widgets.BoundedIntText(description="Meta horas", value=0, min=0, max=10)
main_target_minutes_w = widgets.BoundedIntText(description="Meta minutos", value=45, min=0, max=59)
main_target_seconds_w = widgets.BoundedIntText(description="Meta segs", value=0, min=0, max=59)

# Test races substituindo o campo JSON por três blocos de entrada
test_distance_options = ["None","5K","10K","Half Marathon","Marathon"]
test_name_options = ["Park Run","Corrida local","Prova oficial","Treino controlado","Outro"]
# Teste 1
test1_distance_w = widgets.Dropdown(description="Teste1 dist", options=test_distance_options, value="None")
test1_date_w = widgets.DatePicker(description="Teste1 data")
test1_name_w = widgets.Dropdown(description="Teste1 tipo", options=test_name_options, value="Park Run")
# Teste 2
test2_distance_w = widgets.Dropdown(description="Teste2 dist", options=test_distance_options, value="None")
test2_date_w = widgets.DatePicker(description="Teste2 data")
test2_name_w = widgets.Dropdown(description="Teste2 tipo", options=test_name_options, value="Park Run")
# Teste 3
test3_distance_w = widgets.Dropdown(description="Teste3 dist", options=test_distance_options, value="None")
test3_date_w = widgets.DatePicker(description="Teste3 data")
test3_name_w = widgets.Dropdown(description="Teste3 tipo", options=test_name_options, value="Park Run")

secondary_obj_w = widgets.SelectMultiple(description="Objetivos 2ários", options=UserProfile.SECONDARY_OBJECTIVES_OPTIONS)

# --- Ritmos e zonas ---
zones_method_w = widgets.Dropdown(description="Método zonas", options=[("Jack Daniels", "jack_daniels"), ("Critical Velocity", "critical_velocity")], value="jack_daniels")
# campos separados para tempos recentes
recent_5k_min_w = widgets.BoundedIntText(description="5K min", value=0, min=0, max=180)
recent_5k_sec_w = widgets.BoundedIntText(description="5K seg", value=0, min=0, max=59)
recent_10k_min_w = widgets.BoundedIntText(description="10K min", value=0, min=0, max=180)
recent_10k_sec_w = widgets.BoundedIntText(description="10K seg", value=0, min=0, max=59)
recent_half_min_w = widgets.BoundedIntText(description="21K min", value=0, min=0, max=240)
recent_half_sec_w = widgets.BoundedIntText(description="21K seg", value=0, min=0, max=59)
recent_marathon_min_w = widgets.BoundedIntText(description="42K min", value=0, min=0, max=360)
recent_marathon_sec_w = widgets.BoundedIntText(description="42K seg", value=0, min=0, max=59)

zone_mix_easy_w = widgets.FloatSlider(description="Easy %", value=0.55, min=0.1, max=0.8, step=0.05)
zone_mix_tempo_w = widgets.FloatSlider(description="Tempo %", value=0.25, min=0.05, max=0.7, step=0.05)
zone_mix_interval_w = widgets.FloatSlider(description="Interval %", value=0.20, min=0.05, max=0.6, step=0.05)
vdot_w = widgets.BoundedFloatText(description="VDOT (opc)", value=0, min=0, max=100)

# --- Preferências de sessão ---
intervals_w = widgets.Checkbox(description="Intervalos", value=True)
tempo_w = widgets.Checkbox(description="Tempo run", value=True)
long_run_w = widgets.Checkbox(description="Longão", value=True)
cross_training_w = widgets.Checkbox(description="Cross training", value=False)
variety_w = widgets.Dropdown(description="Variedade", options=["baixa","moderada","alta"], value="moderada")
social_w = widgets.SelectMultiple(description="Opções sociais", options=["grupo","parceiro","sozinho","virtual"])
rpe_w = widgets.IntSlider(description="RPE chave", value=7, min=1, max=10)
long_tol_w = widgets.Dropdown(description="Tolerância longas", options=["baixa","moderada","alta"], value="moderada")
routine_fun_w = widgets.Dropdown(description="Rotina vs diversão", options=["rotina","equilibrado","diversão"], value="equilibrado")

# --- Lesões e restrições ---
prev_inj_w = widgets.SelectMultiple(description="Lesões passadas", options=UserProfile.COMMON_INJURIES)
current_inj_w = widgets.SelectMultiple(description="Lesões atuais", options=UserProfile.COMMON_INJURIES)
triggers_w = widgets.SelectMultiple(description="Gatilhos", options=["descidas","frio","calor","chuva","altitude","piso duro","nenhum"])
red_zones_w = widgets.SelectMultiple(description="Zonas vermelhas", options=["paralelepípedo","descidas","aclive","trilha técnica","estrada de terra","nenhum"])
impact_lim_w = widgets.SelectMultiple(description="Limites impacto", options=["evitar descidas","evitar impacto excessivo","correr em piso macio","nenhum"])
strength_w = widgets.SelectMultiple(description="Rotinas de força", options=["mobilidade quadril","core","força geral","funcional","yoga","nenhum"])

# --- Logística de sessão ---
warmup_w = widgets.BoundedIntText(description="Aquecimento (min)", value=10, min=0, max=60)
cooldown_w = widgets.BoundedIntText(description="Desaquecimento (min)", value=10, min=0, max=60)
commute_w = widgets.BoundedIntText(description="Deslocamento (min)", value=0, min=0, max=90)

form = widgets.Accordion(children=[
    widgets.VBox([name_w, age_w, gender_w, weight_w, height_w, hr_rest_w, hr_max_w]),
    widgets.VBox([years_w, experience_w, current_km_w, avg_km_w, peak_km_w, consistent_days_w, adherence_w, tolerated_w]),
    widgets.VBox([days_week_w, hours_day_w, preferred_time_w, preferred_location_w, preferred_days_w, long_run_days_w, stressful_blocks_select_w, use_alt_weeks_w, alt_blocks_select_w, alt_long_days_select_w, weekly_schedule_select_w]),
    widgets.VBox([main_distance_w, main_date_w, main_name_w, main_city_w, main_target_hours_w, main_target_minutes_w, main_target_seconds_w, test1_distance_w, test1_date_w, test1_name_w, test2_distance_w, test2_date_w, test2_name_w, test3_distance_w, test3_date_w, test3_name_w, secondary_obj_w]),
    widgets.VBox([zones_method_w, recent_5k_min_w, recent_5k_sec_w, recent_10k_min_w, recent_10k_sec_w, recent_half_min_w, recent_half_sec_w, recent_marathon_min_w, recent_marathon_sec_w, zone_mix_easy_w, zone_mix_tempo_w, zone_mix_interval_w, vdot_w]),
    widgets.VBox([intervals_w, tempo_w, long_run_w, cross_training_w, variety_w, social_w, rpe_w, long_tol_w, routine_fun_w]),
    widgets.VBox([prev_inj_w, current_inj_w, triggers_w, red_zones_w, impact_lim_w, strength_w]),
    widgets.VBox([warmup_w, cooldown_w, commute_w])
])
form.set_title(0, "Pessoais")
form.set_title(1, "Experiência")
form.set_title(2, "Disponibilidade")
form.set_title(3, "Objetivos")
form.set_title(4, "Zonas e ritmos")
form.set_title(5, "Preferências de sessão")
form.set_title(6, "Lesões e restrições")
form.set_title(7, "Logística")

display(form)



Accordion(children=(VBox(children=(Text(value='Corredor(a)', description='Nome'), BoundedIntText(value=30, des…

In [25]:

def _safe_json(text, default):
    if not text or text.strip() == "":
        return default
    try:
        return json.loads(text)
    except Exception as exc:
        print(f"JSON inválido ({exc}); usando padrão {default}")
        return default

def _split_text(text):
    if not text:
        return []
    return [item.strip() for item in text.split(',') if item.strip()]



In [26]:
def build_profile_from_widgets():
    # Construir objetivo principal
    # Converter tempo meta
    target_time = None
    if main_target_hours_w.value or main_target_minutes_w.value or main_target_seconds_w.value:
        h = int(main_target_hours_w.value or 0)
        m = int(main_target_minutes_w.value or 0)
        s = int(main_target_seconds_w.value or 0)
        target_time = f"{h:02d}:{m:02d}:{s:02d}"
    main_goal = RaceGoal(
        distance=main_distance_w.value,
        date=main_date_w.value or date.today(),
        name=main_name_w.value,
        location=main_city_w.value,
        is_main_goal=True,
        target_time=target_time,
    )

    # Provas teste
    test_races = []
    for dist, dte, nm in [
        (test1_distance_w.value, test1_date_w.value, test1_name_w.value),
        (test2_distance_w.value, test2_date_w.value, test2_name_w.value),
        (test3_distance_w.value, test3_date_w.value, test3_name_w.value),
    ]:
        if dist and dist != "None":
            test_races.append(
                RaceGoal(
                    distance=dist,
                    date=dte or date.today(),
                    name=nm,
                    location="",
                    is_main_goal=False,
                    target_time=None,
                )
            )

    # Conversão de blocos críticos para dicionário por dia
    def convert_blocks(selected):
        if not selected or "Nenhum" in selected:
            return {}
        mapping = {}
        for slot in selected:
            key = None
            if slot == "Manhã":
                key = "morning"
            elif slot == "Tarde":
                key = "afternoon"
            elif slot == "Noite":
                key = "night"
            if key:
                for day in PlanGenerator.DAYS_OF_WEEK:
                    mapping.setdefault(day, []).append(key)
        return mapping

    stressful_blocks = convert_blocks(stressful_blocks_select_w.value)
    alt_stressful_blocks = convert_blocks(alt_blocks_select_w.value)

    # Conversão de dias longos alternados
    alt_long_run_days = list(alt_long_days_select_w.value)

    # Conversão de grade semanal
    def convert_weekly_schedule(selection):
        if selection == "nenhum" or selection == "flexível":
            return {}
        schedule = {}
        if selection == "manhãs":
            for day in PlanGenerator.DAYS_OF_WEEK:
                schedule.setdefault(day, []).append({"start": "06:00", "end": "07:15", "max_minutes": 60, "surfaces": ["rua"]})
        elif selection == "tardes":
            for day in PlanGenerator.DAYS_OF_WEEK:
                schedule.setdefault(day, []).append({"start": "12:00", "end": "13:15", "max_minutes": 60, "surfaces": ["rua"]})
        elif selection == "noites":
            for day in PlanGenerator.DAYS_OF_WEEK:
                schedule.setdefault(day, []).append({"start": "19:00", "end": "20:15", "max_minutes": 60, "surfaces": ["rua"]})
        return schedule

    weekly_schedule = convert_weekly_schedule(weekly_schedule_select_w.value)

    # Conversão de tempos recentes
    def to_hms(minutes, seconds):
        total_sec = int(minutes or 0) * 60 + int(seconds or 0)
        h = total_sec // 3600
        m = (total_sec % 3600) // 60
        s = total_sec % 60
        return f"{h:02d}:{m:02d}:{s:02d}"

    recent_race_times = {}
    if recent_5k_min_w.value or recent_5k_sec_w.value:
        recent_race_times["5K"] = to_hms(recent_5k_min_w.value, recent_5k_sec_w.value)
    if recent_10k_min_w.value or recent_10k_sec_w.value:
        recent_race_times["10K"] = to_hms(recent_10k_min_w.value, recent_10k_sec_w.value)
    if recent_half_min_w.value or recent_half_sec_w.value:
        recent_race_times["Half Marathon"] = to_hms(recent_half_min_w.value, recent_half_sec_w.value)
    if recent_marathon_min_w.value or recent_marathon_sec_w.value:
        recent_race_times["Marathon"] = to_hms(recent_marathon_min_w.value, recent_marathon_sec_w.value)

    # Construção do dicionário zone_mix
    zone_mix = {
        "easy": float(zone_mix_easy_w.value),
        "tempo": float(zone_mix_tempo_w.value),
        "interval": float(zone_mix_interval_w.value),
    }

    profile = UserProfile(
        name=name_w.value,
        age=int(age_w.value or 0),
        weight_kg=float(weight_w.value or 0),
        height_cm=float(height_w.value or 0),
        gender=gender_w.value,
        hr_resting=int(hr_rest_w.value or 0) or None,
        hr_max=int(hr_max_w.value or 0) or None,
        years_running=float(years_w.value or 0),
        current_weekly_km=float(current_km_w.value or 0),
        average_weekly_km=float(avg_km_w.value or 0),
        recent_peak_weekly_km=float(peak_km_w.value or 0),
        consistent_days_per_week=int(consistent_days_w.value or 0),
        tolerated_workouts=list(tolerated_w.value),
        adherence_score=int(adherence_w.value or 0),
        experience_level=experience_w.value,
        main_race=main_goal,
        test_races=test_races,
        secondary_objectives=list(secondary_obj_w.value),
        days_per_week=int(days_week_w.value),
        hours_per_day=float(hours_day_w.value),
        preferred_time=preferred_time_w.value,
        preferred_location=list(preferred_location_w.value),
        preferred_days=list(preferred_days_w.value),
        stressful_blocks=stressful_blocks,
        long_run_preference_days=list(long_run_days_w.value),
        use_alternating_weeks=bool(use_alt_weeks_w.value),
        alternate_stressful_blocks=alt_stressful_blocks,
        alternate_long_run_days=alt_long_run_days,
        weekly_schedule=weekly_schedule,
        default_warmup_minutes=int(warmup_w.value or 0),
        default_cooldown_minutes=int(cooldown_w.value or 0),
        commute_minutes=int(commute_w.value or 0),
        typical_key_workout_rpe=int(rpe_w.value or 0) if rpe_w.value else None,
        long_session_tolerance=long_tol_w.value,
        variety_preference=variety_w.value,
        social_training_options=list(social_w.value),
        routine_vs_fun_balance=routine_fun_w.value,
        recent_race_times=recent_race_times,
        zones_calculation_method=zones_method_w.value,
        zone_mix_preference=zone_mix,
        vdot_estimate=float(vdot_w.value) if vdot_w.value else None,
        session_preferences={
            "intervals": intervals_w.value,
            "tempo": tempo_w.value,
            "long_run": long_run_w.value,
            "cross_training": cross_training_w.value,
        },
        previous_injuries=list(prev_inj_w.value),
        current_injuries=list(current_inj_w.value),
        injury_triggers=list(triggers_w.value),
        red_zones=list(red_zones_w.value),
        impact_limitations=list(impact_lim_w.value),
        strength_routines=list(strength_w.value),
    )

    return profile



def on_generate_profile(_=None):
    clear_output(wait=True)
    display(form, generate_btn, save_json_btn, plan_btn)
    profile = build_profile_from_widgets()
    # Calcular zonas (se informado)
    zones = None
    if profile.recent_race_times:
        zones = TrainingZones(method=profile.zones_calculation_method)
        for dist, tempo in profile.recent_race_times.items():
            zones.add_race_result(
                distance_km=TrainingZones._distance_from_label(dist),
                time_str=tempo,
            )
    # Determinar categoria com base em experience_level
    classification = profile.experience_level
    profile_dict = profile.to_dict()
    print("Resumo do perfil gerado:")
    print(profile)
    print(f"Categoria: {classification}")
    print("Generator params:", profile.to_generator_params())
    if zones:
        print("Zonas calculadas:")
        print(zones)
    # Guardar
    on_generate_profile.profile_cache = profile_dict
    on_generate_profile.classification = classification


def on_save_json(_=None):
    profile_dict = getattr(on_generate_profile, "profile_cache", None)
    classification = getattr(on_generate_profile, "classification", None)
    if not profile_dict:
        on_generate_profile()
        profile_dict = getattr(on_generate_profile, "profile_cache", None)
        classification = getattr(on_generate_profile, "classification", None)
    if not profile_dict:
        print("Nenhum perfil disponível para salvar.")
        return
    if classification:
        profile_dict["classification"] = classification
    filename = f"perfil_corredor_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(profile_dict, f, indent=2, ensure_ascii=False)
    print(f"Perfil salvo em {filename}")


def on_generate_plan(_=None):
    clear_output(wait=True)
    display(form, generate_btn, save_json_btn, plan_btn)
    profile = build_profile_from_widgets()
    zones = None
    if profile.recent_race_times:
        zones = TrainingZones(method=profile.zones_calculation_method)
        for dist, tempo in profile.recent_race_times.items():
            zones.add_race_result(
                distance_km=TrainingZones._distance_from_label(dist),
                time_str=tempo,
            )
    plan = PlanGenerator.generate_plan(
        name=f"Plano {profile.main_race.distance}",
        goal=profile.main_race.distance,
        level=profile.experience_level,
        days_per_week=profile.days_per_week,
        training_zones=zones,
        user_profile=profile,
    )

    visual_plan = plan.to_visual_str(show_all_weeks=True)
    print("\n🎉 Plano gerado! Veja os detalhes ilustrados abaixo:\n")
    print(visual_plan)

# Botões de ação
generate_btn = widgets.Button(description="Gerar perfil + JSON", button_style="primary")
save_json_btn = widgets.Button(description="Salvar JSON", button_style="success")
plan_btn = widgets.Button(description="Gerar plano", button_style="info")

generate_btn.on_click(on_generate_profile)
save_json_btn.on_click(on_save_json)
plan_btn.on_click(on_generate_plan)

# Exibir botões
try:
    display(widgets.HBox([generate_btn, save_json_btn, plan_btn]))
except Exception:
    pass


Accordion(children=(VBox(children=(Text(value='Corredor(a)', description='Nome'), BoundedIntText(value=30, des…

Button(button_style='primary', description='Gerar perfil + JSON', style=ButtonStyle())

Button(button_style='success', description='Salvar JSON', style=ButtonStyle())

Button(button_style='info', description='Gerar plano', style=ButtonStyle())


🎉 Plano gerado! Veja os detalhes ilustrados abaixo:


🏃‍♂️ PLANO DE TREINO: Plano 10K
🎯 Meta: 10K
📊 Nível: Intermediate
📅 Duração: 4 semanas
🗓️  Dias de treino: 4 dias/semana
📏 Kilometragem total: 115.0km

📅 SEMANA 1 | 📏 40km total
💡 Fase: Base (semana 1/4)
Welcome to your training plan! Start easy and focus on consistency.

💬 Feedback semanal: Plano em modo conservador (regra dos 10%, semanas de recuperação). Compartilhe feedback semanal sobre dor/fadiga para ajustes baseados em evidências.
  📍 Monday: 😴 Descanso
  ┈┈┈┈┈┈┈┈┈┈┈┈
  📍 Tuesday: 🟢 Fácil
     🧭 Resumo: 10km | @ 4:22/km | Zona easy | ⏱️ 1h
     📝 Notas: Ritmo confortável, esforço conversacional
  ┈┈┈┈┈┈┈┈┈┈┈┈
  📍 Wednesday: 😴 Descanso
  ┈┈┈┈┈┈┈┈┈┈┈┈
  📍 Thursday: 🟢 Fácil
     🧭 Resumo: 10km | @ 4:22/km | Zona easy | ⏱️ 1h
     📝 Notas: Ritmo confortável, esforço conversacional
  ┈┈┈┈┈┈┈┈┈┈┈┈
  📍 Friday: 🟢 Fácil
     🧭 Resumo: 5km | @ 4:22/km | Zona easy | ⏱️ 50min
     📝 Notas: Ritmo confortável, esforço conversacional
  ┈┈